## Preambule

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path
import plotly.graph_objects as go
import xarray as xr
from plotly.subplots import make_subplots
from scipy.stats import spearmanr

In [2]:
path = Path.cwd()

## Input

Variable lists

In [3]:
XRraw = xr.open_dataset(path / "Data/XRdata.nc")
Variables = np.array(XRraw.Variable)

In [4]:
Varlist = pd.read_excel(path / "Data/Varlist.xlsx", sheet_name = "Data")
vars = np.array(Varlist["Variable"])
frac = np.array(Varlist["Fractional"]).astype(str)
unifrac = frac[frac != 'nan']
cats = np.array(Varlist["Category"])

newvars = list(vars)
newcats = list(cats)
for f in unifrac:
    vrs = vars[frac == f]
    cts = cats[frac == f]
    for v_i, v in enumerate(vrs):
        newvars.append(v+' (fr)')
        newcats.append(cts[v_i]+' (fr)')
vars = np.array(newvars).astype(str)
cats = np.array(newcats).astype(str)
Names = np.unique(cats)
Vlists = []
for i in range(len(Names)):
    Vlists.append(np.unique(vars[cats == Names[i]]))

General data

In [5]:
DF = pd.read_csv(path / "Data/Models.csv")
DF_counts = pd.read_csv(path / "Data/Counts.csv", index_col=0)

Detailed output

In [6]:
XRraw = xr.open_dataset(path / "Data/XRdata.nc")
XRmeta = xr.open_dataset(path / "Data/XRmeta.nc")

In [7]:
modscens = np.array(XRraw.ModelScenario)
mods = np.array([i.split('|')[0] for i in modscens])
ccat = np.array(XRmeta.sel(ModelScenario=XRraw.ModelScenario).Category.data)
unimods = np.unique(mods)
uniccat = np.unique(ccat)

Clean data

In [8]:
modscen_new = []
for i in range(len(mods)):
    m = mods[i]
    c = ccat[i]
    if len(np.where(mods == m)[0]) >= 10 and c != "C8":
        modscen_new.append(modscens[i])
XRraw = XRraw.sel(ModelScenario = modscen_new)

## Correlations

In [9]:
def correlation(varval, models, ccats):
    rho_m = spearmanr(varval, models)
    rho_c = spearmanr(varval, models)

## Plot

In [10]:
def boxplots(Var, Year):
    XR_filt = XRraw.sel(Variable=Var)
    XR_filt = XR_filt.dropna('ModelScenario', how='any')
    ModelScenarios = np.array(XR_filt.ModelScenario)
    Models = np.array([XR_filt.ModelScenario.data[i].split('|')[0] for i in range(len(XR_filt.ModelScenario))])
    Ccat = np.array(XRmeta.sel(ModelScenario=XR_filt.ModelScenario).Category.data)
    unimodels = np.unique(Models)
    unicats = np.unique(Ccat)
    Perc_models = np.zeros(shape=(len(unimodels)))
    Perc_cats = np.zeros(shape=(len(unicats)))
    for m in range(len(unimodels)):
        ms = ModelScenarios[Models == unimodels[m]]
        #Perc_models[m] = np.percentile(np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value), 50)
        Perc_models[m] = np.median(np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value))
    for c in range(len(unicats)):
        ms = ModelScenarios[Ccat == unicats[c]]
        #Perc_cats[c] = np.percentile(np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value), 50)
        Perc_cats[c] = np.median(np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value))

    cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
            'magenta', 'red', 'darkgrey', 'blue', 'black', 'darkgreen']
            
    medians = [np.median(np.array(XR_filt.sel(Time=Year).Value)[Models == i]) for i in unimodels]
    ranks_m = [np.argsort(np.argsort(medians))[np.where(unimodels == i)[0][0]] for i in Models]
    ranks_c = [int(i[1]) for i in Ccat]

    r1, p1 = spearmanr(ranks_m, np.array(XR_filt.sel(Time=Year).Value))
    r2, p2 = spearmanr(ranks_c, np.array(XR_filt.sel(Time=Year).Value))
    r1 = r1*np.sign(r1)
    r2 = r2*np.sign(r2)
    fig = make_subplots(rows=3, cols=2, subplot_titles=("",#"Spearman's rank correlation: "+str(np.round(r1, 2))+' (p='+str(np.round(p1, 3))+')',
                                                        "",#"Spearman's rank correlation: "+str(np.round(r2, 2))+' (p='+str(np.round(p2, 3))+')',
                                                        ""),
                        specs=[[{"rowspan": 2}, {"rowspan": 2}], [None, None], [{"colspan": 2}, None]], horizontal_spacing = 0.025, column_widths=[0.33, 0.33], vertical_spacing=0.05)

    for m in range(len(unimodels)):
        m2 = np.argsort(Perc_models)[m]
        col = cols[m2]
        ms = ModelScenarios[Models == unimodels[m2]]
        y = np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value)
        fig.add_trace(go.Box(y=y, name=unimodels[m2], text=ms, fillcolor=col, line_color='black', marker=dict(color=col), boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.2, marker_size=3, line_width=1), 1, 1)
    for m in range(len(unicats)):
        rgb = mpl.colors.colorConverter.to_rgb(plt.cm.get_cmap('RdBu_r')(0.+m/len(unicats)))
        col = 'rgb('+str(rgb[0])+','+str(rgb[1])+','+str(rgb[2])+')'
        ms = ModelScenarios[Ccat == unicats[m]]
        y = np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value)
        fig.add_trace(go.Box(y=y, name=unicats[m], text=ms, fillcolor=col, line_color='black', marker=dict(color=col), boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.2, marker_size=3, line_width=1), 1, 2)
    for m in range(len(unimodels)):
        m2 = np.argsort(Perc_models)[m]
        col = cols[m2]
        for c in range(len(unicats)):
            ms = ModelScenarios[(Ccat == unicats[c]) & (Models == unimodels[m2])]
            y = np.array(XR_filt.sel(Time=Year, ModelScenario=ms).Value)
            fig.add_trace(go.Box(y=y, name=unicats[c]+' '+unimodels[m2], text=ms, fillcolor=col, line_color='black', marker=dict(color=col), boxpoints=False, jitter=0.5, pointpos=-1.8, whiskerwidth=0.2, marker_size=3, line_width=1), 3, 1)
    #fig.update_yaxes(title=Var+' in '+str(Year), row=1, col=1)
    fig.update_xaxes(tickangle=0, tickvals=unimodels[np.argsort(Perc_models)], ticktext=['<br>-'.join(i.split(' ')[0].split('-')) for i in unimodels[np.argsort(Perc_models)]], tickfont=dict(size=11), row=1, col=1)
    fig.update_layout(
        title=Var+' in '+str(Year),
        yaxis=dict(
            autorange=True,
            showgrid=True,
            zeroline=True,
            gridcolor='rgb(255, 255, 255)',
            gridwidth=1,
            zerolinecolor='rgb(255, 255, 255)',
            zerolinewidth=2,
        ),
        #paper_bgcolor='rgb(243, 243, 243)',
        #plot_bgcolor='rgb(243, 243, 243)',
        showlegend=False
    )
    fig.update_layout({'plot_bgcolor':'rgb(243, 243, 243)'})
    fig.update_yaxes(ticktext=['']*len(np.arange(0, 270, 25)), row=1, col=2)
    #fig.update_yaxes(title=Var+' in '+str(Year), row=3, col=1)
    fig.update_layout(height=1000, width = 1500)
    return fig

In [11]:
def save_html(fg, varh, var, y, path):
    var2 = 'o_'.join('_'.join('_'.join(var.split('|')).split('/ ')).split('/o '))
    name = var2+'_'+str(y)+'.html'
    folderpath = path / 'Figures/Boxplots' / varh
    folderpath.mkdir(parents=True, exist_ok=True)
    try:
        os.remove(folderpath / name)
    except:
        3
    def html_w(typ):
        return '<html> '+typ+' <p style="font-family: Arial">'

    with open(folderpath / name, 'a') as f:
        f.write(html_w('<h1>')+var+'</p></h1>')
        f.write(fg.to_html(full_html=False, include_plotlyjs='cdn'))

def save_html_group(figs, N):
    name = Names[N]
    try:
        os.remove('Figures/Boxplots/'+name+'.html')
    except:
        3
    def html_w(typ):
        return '<html> '+typ+' <p style="font-family: Arial">'

    with open('Figures/Boxplots/'+name+'.html', 'a') as f:
        f.write(html_w('<h1>')+'Primary Energy boxplots</p></h1>')
        f.write(html_w('<body>')+'Box plots for 2100. </p></body>')

        for n_i in range(len(figs)):
            if n_i > 0:
                f.write('<hr>')
            f.write(html_w('<h1>')+'Fig. '+str(n_i+1)+' - '+Vlists[N][n_i]+'</p></h1>')
            f.write(figs[n_i].to_html(full_html=False, include_plotlyjs='cdn'))

In [12]:
for n_i, n in enumerate(Vlists):
    figs = []
    for v in n:
        for y in [2030, 2050, 2100]:
            #print(v, y)
            fig = boxplots(v, y)
            figs.append(fig)
            folderpath = path / 'Figures/Boxplots' / Names[n_i]
            var2 = 'o_'.join('_'.join('_'.join(v.split('|')).split('/ ')).split('/o '))
            name = var2+'_'+str(y)+'.svg'
            fig.write_image(folderpath / name)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was de

In [13]:
for n_i, n in enumerate(Vlists):
    figs = []
    for v in n:
        for y in [2030, 2050, 2100]:
            #print(v, y)
            fig = boxplots(v, y)
            figs.append(fig)
            save_html(fig, Names[n_i], v, y, path)
    #save_html_group(figs, n_i)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1096\1505205397.py:43: MatplotlibDeprecationWarning:

The get_cmap function was de